# The University of Melbourne, School of Computing and Information Systems
# COMP90049 Introduction to Machine Learning, 2020 Semester 1
-----
## Project 1: Understanding Student Success with Naive Bayes
-----
###### Student Name(s): Youngsun Kim
###### Python version: 3.7
###### Submission deadline: 11am, Wed 22 Apr 2020

This iPython notebook is a template which you will use for your Project 1 submission. 

Marking will be applied on the five functions that are defined in this notebook, and to your responses to the questions at the end of this notebook.

You may change the prototypes of these functions, and you may write other functions, according to your requirements. We would appreciate it if the required functions were prominent/easy to find. 


In [1]:
import numpy as np
from sklearn.model_selection import KFold
import pandas as pd
import time

# Global variables
data_total = [0]
d_feature = [0]
d_class = [0]
count_df = [0]

In [2]:
# This function should open a data file in csv, and transform it into a usable format
def load_data(filepath):
    global data_total, d_feature, d_class
    data_total = pd.read_csv(filepath, sep=',')
    d_feature = data_total.iloc[:, 0:29]
    d_class = data_total['Grade']
    return

In [3]:
# This function should split a data set into a training set and hold-out test set
# res[0] :  training data [0-518] 80% of the total data
# res[1] : test data [519-648] 20% of the total data
def split_data():
    arr_train = list(range(0, 519))
    arr_test = list(range(519, 649))
    res = [0]
    res[0] = arr_train
    res.append(arr_test)

    return res


In [4]:
# This function should build a supervised NB model
def train(train_index):
    global d_feature
    global d_class
    global data_total

    train_inst = data_total.iloc[train_index]

    # region [make count table]
    global count_df

    count_df[0] = train_inst.groupby(['Grade', 'school']).size().reset_index(name='count')

    # test
    # print("[count_school]"); print(count_df[0])
    # access to conditional value
    # res = count_df[0].loc[(count_df[0]['Grade'] == 'A') & (count_df[0]['school'] == 'GP')].at[0,'count']
    # print("res", res)
    ##
    count_df.append(train_inst.groupby(['Grade', 'sex']).size().reset_index(name='count'))
    count_df.append(train_inst.groupby(['Grade', 'address']).size().reset_index(name='count'))
    count_df.append(train_inst.groupby(['Grade', 'famsize']).size().reset_index(name='count'))
    count_df.append(train_inst.groupby(['Grade', 'Pstatus']).size().reset_index(name='count'))

    count_df.append(train_inst.groupby(['Grade', 'Medu']).size().reset_index(name='count'))
    count_df.append(train_inst.groupby(['Grade', 'Fedu']).size().reset_index(name='count'))
    count_df.append(train_inst.groupby(['Grade', 'Mjob']).size().reset_index(name='count'))
    count_df.append(train_inst.groupby(['Grade', 'Fjob']).size().reset_index(name='count'))
    count_df.append(train_inst.groupby(['Grade', 'reason']).size().reset_index(name='count'))

    count_df.append(train_inst.groupby(['Grade', 'guardian']).size().reset_index(name='count'))
    count_df.append(train_inst.groupby(['Grade', 'traveltime']).size().reset_index(name='count'))
    count_df.append(train_inst.groupby(['Grade', 'studytime']).size().reset_index(name='count'))
    count_df.append(train_inst.groupby(['Grade', 'failures']).size().reset_index(name='count'))
    count_df.append(train_inst.groupby(['Grade', 'schoolsup']).size().reset_index(name='count'))

    count_df.append(train_inst.groupby(['Grade', 'famsup']).size().reset_index(name='count'))
    count_df.append(train_inst.groupby(['Grade', 'paid']).size().reset_index(name='count'))
    count_df.append(train_inst.groupby(['Grade', 'activities']).size().reset_index(name='count'))
    count_df.append(train_inst.groupby(['Grade', 'nursery']).size().reset_index(name='count'))
    count_df.append(train_inst.groupby(['Grade', 'higher']).size().reset_index(name='count'))

    count_df.append(train_inst.groupby(['Grade', 'internet']).size().reset_index(name='count'))
    count_df.append(train_inst.groupby(['Grade', 'romantic']).size().reset_index(name='count'))
    count_df.append(train_inst.groupby(['Grade', 'famrel']).size().reset_index(name='count'))
    count_df.append(train_inst.groupby(['Grade', 'freetime']).size().reset_index(name='count'))
    count_df.append(train_inst.groupby(['Grade', 'goout']).size().reset_index(name='count'))

    count_df.append(train_inst.groupby(['Grade', 'Dalc']).size().reset_index(name='count'))
    count_df.append(train_inst.groupby(['Grade', 'Walc']).size().reset_index(name='count'))
    count_df.append(train_inst.groupby(['Grade', 'health']).size().reset_index(name='count'))
    count_df.append(train_inst.groupby(['Grade', 'absences']).size().reset_index(name='count'))
    count_df.append(train_inst.groupby('Grade').size().reset_index(name='count'))  # 29

    # print(count_df)
    # test

    # print("모든 count table")
    # print(count_df)
    # print("[count_Grade]")
    # print(count_df[29])
    # print("[Grade at : ]", count_df[29].iat[0, 1])

    # endregion

    return


In [5]:
# This function should predict the class for an instance or a set of instances, based on a trained model
def predict(test_index):
    GradeArr = ["A+", "A", "B", "C", "D", "F"]
    AttbArr = ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian',
               'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher',
               'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

    test_inst = data_total.iloc[test_index]
    grade_tot = count_df[29].sum(numeric_only=True).iat[0]
    predicted_class = [0]
    i = 0

    for inst in test_inst.iterrows():

        max_y = float('-inf')  # minimum int
        est_res = ""

        for g in GradeArr:

            grade_cnt = count_df[29].loc[count_df[29]['Grade'] == g].iat[0, 1]
            est_y = np.log(grade_cnt / grade_tot)

            for att_idx in range(29):
                df_condition = count_df[att_idx].loc[
                    (count_df[att_idx]['Grade'] == g) & (count_df[att_idx][AttbArr[att_idx]] == inst[1].array[att_idx])]

                if df_condition.empty == True:
                    cnt_test = 0  # 0 - smoothing?
                else:
                    cnt_test = df_condition.iat[0, 2]

                if grade_cnt <=0:
                    est_y += 0
                if cnt_test <=0:
                    est_y = float('-inf')+1
                else:
                    est_y += np.log(cnt_test / grade_cnt)

            if est_y >= max_y:
                max_y = est_y
                est_res = g

        if i == 0:
            predicted_class[0] = est_res
        else:
            predicted_class.append(est_res)
        i += 1

    return predicted_class


In [6]:
#For Q5 Only, commented out at the main function
def predict_fair(test_index):
    GradeArr = ["A+", "A", "B", "C", "D", "F"]
    AttbArr = ['school', 'sex', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu', 'Mjob', 'Fjob', 'reason', 'guardian',
               'traveltime', 'studytime', 'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher',
               'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences']

    test_inst = data_total.iloc[test_index]
    grade_tot = count_df[29].sum(numeric_only=True).iat[0]
    predicted_class = [0]
    i = 0

    for inst in test_inst.iterrows():

        max_y = float('-inf')  # minimum int
        est_res = ""

        for g in GradeArr:

            grade_cnt = count_df[29].loc[count_df[29]['Grade'] == g].iat[0, 1]
            est_y = np.log(grade_cnt / grade_tot)


            for att_idx in range(29-7):

                df_condition = count_df[att_idx].loc[
                    (count_df[att_idx]['Grade'] == g) & (count_df[att_idx][AttbArr[att_idx]] == inst[1].array[att_idx])]

                if df_condition.empty == True:
                    cnt_test = 0  # 0 - smoothing?
                else:
                    cnt_test = df_condition.iat[0, 2]

                # remove 7 features 1 4 5 6 7 8 10
                if att_idx == 1 or att_idx == 4 or att_idx == 5 or att_idx == 6 or att_idx == 7 or att_idx == 8 or att_idx == 10 or grade_cnt == 0 or grade_cnt == 0.0:
                    est_y += 0
                else:
                    est_y += np.log(cnt_test / grade_cnt)

            if est_y >= max_y:
                max_y = est_y
                est_res = g

        if i == 0:
            predicted_class[0] = est_res
        else:
            predicted_class.append(est_res)
        i += 1

    return predicted_class


In [7]:
# This function should evaluate a set of predictions in terms of accuracy
def evaluate(test_index, prd_res):
    real_res = d_class.iloc[test_index]
    tot_cnt = real_res.size
    i = 0
    correct_cnt = 0

    for real in real_res:
        if real == prd_res[i]:
            correct_cnt += 1
            # print("correct index: ", i)
        # else : print("incorrect index: ", i)
        i += 1

    accr = correct_cnt / tot_cnt

    return accr


In [8]:
# region [Main]
load_data('./Data/student.csv')

# region [HOLD-OUT // SPLIT 80:20 // Accuracy : 0.34]
res = split_data()
train(res[0])
prd_res = predict(res[1])  # returns predicted array (test instances')
accr_res = evaluate(res[1], prd_res)
print("Holdout Accuracy: ", accr_res)
# endregion


# region [CROSS-VALIDATION // Accuracy : 0.49]
start_time = time.time()

N_SPLIT = 65
kf = KFold(n_splits=N_SPLIT, random_state=0)  # n_fold=65, 649

i = 0
accr_df = [0]

for train_index, test_index in kf.split(data_total):
    train(train_index)
    prd_res = predict(test_index)

    #(Q5)Fair way
    #prd_res = predict_fair(test_index)

    accr_res = evaluate(test_index, prd_res)
    if i == 0:
        accr_df[i] = accr_res
    else:
        accr_df.append(accr_res)
        
    if i == 64 : print(i+1,"/",N_SPLIT,"Done!")
    else : print(i+1,"/",N_SPLIT,"Processing...")
    
    i += 1

print("[Cross-validation Result]")
print("[Running Time]: ", (time.time() - start_time))
print("[N_SPLIT]: ", N_SPLIT)
print("[Accuracy Array]: ", accr_df)
print("[Averaged Accuracy]: ", np.mean(accr_df))

# endregion


# endregion



Holdout Accuracy:  0.18461538461538463
1 / 65 Processing...
2 / 65 Processing...
3 / 65 Processing...
4 / 65 Processing...
5 / 65 Processing...
6 / 65 Processing...
7 / 65 Processing...
8 / 65 Processing...
9 / 65 Processing...
10 / 65 Processing...
11 / 65 Processing...
12 / 65 Processing...
13 / 65 Processing...
14 / 65 Processing...
15 / 65 Processing...
16 / 65 Processing...
17 / 65 Processing...
18 / 65 Processing...
19 / 65 Processing...
20 / 65 Processing...
21 / 65 Processing...
22 / 65 Processing...
23 / 65 Processing...
24 / 65 Processing...
25 / 65 Processing...
26 / 65 Processing...
27 / 65 Processing...
28 / 65 Processing...
29 / 65 Processing...
30 / 65 Processing...
31 / 65 Processing...
32 / 65 Processing...
33 / 65 Processing...
34 / 65 Processing...
35 / 65 Processing...
36 / 65 Processing...
37 / 65 Processing...
38 / 65 Processing...
39 / 65 Processing...
40 / 65 Processing...
41 / 65 Processing...
42 / 65 Processing...
43 / 65 Processing...
44 / 65 Processing...
45

In [ ]:
## Questions (you may respond in a cell or cells below):
You should respond to Question 1 and two additional questions of your choice.
 A response to a question should take about 100–250 words, and make reference to the data wherever possible.

### Question 1: Naive Bayes Concepts and Implementation
- a Explain the ‘naive’ assumption underlying Naive Bayes. (1) Why is it necessary? (2) Why can it be problematic? Link your discussion to the features of the students data set. [no programming required]
- b Implement the required functions to load the student dataset, and estimate a Naive Bayes model. Evaluate the resulting classifier using the hold-out strategy, and measure its performance using accuracy.
- c What accuracy does your classifier achieve? Manually inspect a few instances for which your classifier made correct predictions, and some for which it predicted incorrectly, and discuss any patterns you can find.

[1-a-(1)] The ‘naive’ assumption underlying Naive Bayes is the conditional independence assumption which is assuming that all the features are assumed to be independent. This assumption is necessary to make the model more feasible. As it is shown below, this assumption makes the joint model into a ‘countable’ and ‘separated’ probabilistic model.
P(x1, x2, ..., xM|y)P(y) => P(x1|y)P(x2|y)...P(xM|y)P(y)


[1-a-(2)] For instance, both parents’ education and jobs can affect to choose a school and the area where the family lives. Real-world data’s features are related to each other. For another example, travel time must be affected by the distance between their schools and address(home) and it can affect study time as well. Also, these features are contributing to the result (Grade) with different weights. The conditional independence assumption is unrealistic as it ignores the correlations between features.


[1-b] In my code, “# region [Main]” has “# region [HOLD-OUT]” and “# region [CROSS-VALIDATION]”. If we look at HOLD-OUT Region for this question, it calls

	load_data('./Data/student.csv') 
	
    res = split_data() : Since using data that has been used for training to test cannot evaluate the classifier’s performance, I used the hold-out method which separates test and training data. Also I used the commonly used rate 80:20 in the function. Thus the training data range becomes [0-518] and the test data becomes [519-648]. At here, I assumed that the both test and training data sets have the same class distribution.
	
    train(res[0])
	
    prd_res = predict(res[1]) : This function returns predicted class array “prd_res”. For estimated y, I moved the equation from [P(y) * πP(x|y)] to log space [logP(y) + ∑logP(x|y)]. Even if there is an unseen case, it would not make the whole equation 0 by multiplying it. Therefore, I did not use any smoothing strategy.
	
    accr_res = evaluate(res[1], prd_res) : For evaluation, I used accuracy. If this classifier would be used as to accept students for a college, I could consider implementing the Precision method. As calculating A/A+ -> A/A+ as True Positive and A/A+ -> Lower than B as False Positive.

[1-c] I got an accuracy of 0.34.
I analyzed instances on an excel file. View online or please find the attached file. https://docs.google.com/spreadsheets/d/10nAbiyw6Uttmpd9PhWgdBuPxSI1iPDpHMKVaGKjizs8/edit?usp=sharing

As the file shows, incorrect results tend to show two characteristics. Firstly, if there is a zero counted feature (unseen feature) for a specific attribute, it tends to classified as a class that has the biggest possibility number in that attribute. 
For example, if you see the cells [O 27:33] of the excel file, the attribute “failures” has unseen feature A+ and B: 
    # P(failures=low|A+) = 0, P(failures=low|B) = 0
    # P(failures=low|F) = 0.35 
In that attribute, P(failures=low|F) = 0.35 has the largest possibility. And this instance is classified as F. 
For another example, cells [AC 44:50] show as below. It has an unseen feature and the largest possibility case is A in attribute “health”. And it is classified as A.
	# P(health=4|A+) = 0
    # P(health=4|A) = 0.22

Secondly, in certain attributes, the wrongly classified class tends to show a large number of observations. And especially if the class has a small number of the total cases, it leads the probability makes a prominent peak (relatively large number / relatively small number). For example,
	# P(A) = 50, P(F) = 82 
At [B 55:60], these Incorrectly estimated classes have a small total number.
	# P(school=MS|F) = P(58/82)  
At [B 31:33], P(MS|real class)-P(MS|estimated class) = 0.3
0.3 is a big difference and it gives “F” win.
	# P(famsize=GT3|F) = P(66/82)  
		At [E 31:33], this case has the big difference = 0.11 again.
    # [L50][P50][V50][X50][Y50] cells show that P(estimated case A) – P(real case B) has a big difference and it is estimated as A.





### Question 3: Training Strategies 
There are other evaluation strategies, which tend to be preferred over the hold-out strategy you implemented in Question 1.
- a Select one such strategy, (i) describe how it works, and (ii) explain why it is preferable over hold-out evaluation. [no programming required]
- b Implement your chosen strategy from Question 3a, and report the accuracy score(s) of your classifier under this strategy. Compare your outcomes against your accuracy score in Question 1, and explain your observations in the context of your response to question 3a.

[3-a-(i)] The cross-validation strategy is preferred over the hold-out. It divides N observation into k sets, and each set has N/k observation. A single set is to be a test data set and k-1 sets are training data sets. It iterates k times and uses each of the k sets as a test data set and the rest of them as training data sets. 
[3-a-(ii)] This strategy uses every instance as a test instance and a training instance. It can help to solve overfitting, selection bias and fluctuation or error by using all the instances as a training and test instance.

[3-b]
I implemented the cross-validation strategy in the main area of my code. The results are as below.

-------------------------------------------------------------------
N_SPLIT:  5

Running time:  130 sec

[Accuracy Array]: [0.35, 0.53, 0.53, 0.46, 0.51]

[Averaged Accuracy]:  0.48
-------------------------------------------------------------------
N_SPLIT:  10

Running time:  136 sec

[Accuracy Array]: [0.28, 0.48, 0.55, 0.48, 0.51, 0.45, 0.45, 0.45, 0.54, 0.5]

[Averaged Accuracy]:  0.47
-------------------------------------------------------------------
N_SPLIT:  65

Running time:  133 sec

[Averaged Accuracy]:  0.49
-------------------------------------------------------------------
Running time:  176

N_SPLIT:  649

[Averaged Accuracy]:  0.48
-------------------------------------------------------------------
I tried 4 cases with N_SPLIT = 5,10,65 and 649(leave-one-out). Each case showed accuracy from 0.47 to 0.49 which is relatively similar to each other. And when we compare the result with the Hold-out’s accuracy 0.34, we can see that 0.34 was a biased result just by the selected test set. The N_SPLIT=10 case explains this problem. The accuracy result [0.28, 0.48, 0.55, 0.48, 0.51, 0.45, 0.45, 0.45, 0.54, 0.5] has 0.28 and it shows that if I just selected that set as a test set for the hold-out strategy, I would get 0.28 for a result and that would be very biased and not an accurate result. In conclusion, the cross-validation can help to mitigate bias and get more accurate evaluation result by averaging multiple results.





### Question 5: Bias and Fairness in Student Success Prediction
As machine learning practitioners, we should be aware of possible ethical considerations around the
applications we develop. The classifier you developed in this assignment could for example be used
to classify college applicants into admitted vs not-admitted – depending on their predicted grade.
- a Discuss ethical problems which might arise in this application and lead to unfair treatment of the applicants. Link your discussion to the set of features provided in the students data set. [no programming required]
- b Select ethically problematic features from the data set and remove them from the data set. Use your own judgment (there is no right or wrong), and document your decisions. Train your Naive Bayes classifier on the resulting data set containing only ‘unproblematic’ features. How does the performance change in comparison to the full classifier?
- c The approach to fairness we have adopted is called “fairness through unawareness” – we simply deleted any questionable features from our data. Removing all problematic features does not guarantee a fair classifier. Can you think of reasons why removing problematic features is not enough? [no programming required]

[5-a] Accepting students based on their gender or their parent’s status (since those features affect calculating Grades in this classifier) can cause discrimination. Features “M/Fjob, M/Fedu, guardian, Pstatus” can lead a consequence that the college selects students based on the presence of parents, their financial status and their education level. Also, the feature “sex” can add a gender preference.


[5-b] At the function predict_fair() in my code, I removed 7 feature columns as I mentioned at [5-a]. It showed an accuracy result of 0.41 which is 0.08 under compared to the original result 0.49. As removing 7 attributes out of 29, the classifier got a deterioration by 16 per cent ((0.49-0.41)*100/0.49).


[5-c] First of all, excluding the controversial feature can be another discrimination. For instance, in our dataset, Female has 44 and male has 18 for A and A+ Grade. If we simply get rid of this observation, it would not be fair to female students. Also, it is ignoring real-life data with social bias and it’s not reflecting the truth truly. Secondly, even if we get rid of the problematic attribute, by ‘red-lining effect’[1], classifiers detect other features that correlate with the problematic attribute, learn the pattern and make a similar estimation. In the paper[1], the income classifier showed a similar result even without the sex attribute. The classifier used sex-correlated other features and discriminated indirectly. In conclusion, removing possible-discriminating features cannot solve the unfairness of machine learning completely.


[Reference]
[1] Toon Calders · Sicco Verwer. Three naive Bayes approaches for discrimination-free classification. [Online]: https://link.springer.com/content/pdf/10.1007/s10618-010-0190-x.pdf



